### Prediccions amb el model

Si volem aprofitar el model per fer prediccions, li hem de passar els paràmetres amb els quals s'ha entrenat el model (dins de `X_train`). Per nosaltres és més fàcil escriure el nom dels jugadors que juguen. A partir d'aquí, crearem la llista de valors dels jugadors i en farem la predicció amb el model.

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import joblib

from sklearn import preprocessing

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Llegim les estadístiques de cada jugador
stats_xr = xr.open_dataset('stats.nc', engine='scipy')
stats_xr

<xarray.Dataset> Size: 116kB
Dimensions:                (matchday: 46, player: 17)
Coordinates:
  * matchday               (matchday) int32 184B 0 1 2 3 4 5 ... 41 42 43 44 45
  * player                 (player) object 136B 'Alejandro' 'Antía' ... 'Víctor'
Data variables: (12/23)
    GamesPlayed            (matchday, player) int32 3kB ...
    PlayedAttack           (matchday, player) int32 3kB ...
    PlayedDefense          (matchday, player) int32 3kB ...
    WinPlayed              (matchday, player) float64 6kB ...
    WinPlayedAttack        (matchday, player) float64 6kB ...
    WinPlayedDefense       (matchday, player) float64 6kB ...
    ...                     ...
    ReceivedDefensePlayed  (matchday, player) float64 6kB ...
    ELOAttack              (matchday, player) float64 6kB ...
    ELODefense             (matchday, player) float64 6kB ...
    AttackIndex            (matchday, player) float64 6kB ...
    DefenseIndex           (matchday, player) float64 6kB ...
    WeigthedELO            (matchday, player) float64 6kB ...

In [3]:
# Llegim l'scaler
scaler = joblib.load('scaler.pkl')
encoder_scores = joblib.load('encoder_scores.pkl')

In [4]:
# Paràmetres que considerem al model, en funció de si el jugador és atacant o defensor (COMPROVAR QUE ÉS EL MATEIX QUE A L'ALTRE NOTEBOOK)
considered_stats_defense = ['GamesPlayed', 'WeigthedELO', 'PlayedDefense', 'WinPlayedDefense', 'ScoredDefensePlayed', 'ReceivedDefensePlayed', 'ELODefense', 'DefenseIndex']
considered_stats_attack = ['GamesPlayed', 'WeigthedELO', 'PlayedAttack', 'WinPlayedAttack', 'ScoredAttackPlayed', 'ReceivedAttackPlayed', 'ELOAttack', 'AttackIndex']

In [5]:
# Si volem carregar el model
model = keras.models.load_model('sequential.keras')

In [6]:
# Jugadors, per alineació
player_list = ['Víctor', 'Elena', 'Guille', 'Luis']

# Creem la llista d'estadístiques per fer la predicció
stats_match = []
for player in player_list:
        if player in [player_list[0], player_list[2]]: # defensors
            # Triem les estadístiques més recents dels jugadors
            player_stats = stats_xr.isel(matchday=-1).sel(player=player)[considered_stats_defense].to_array().values
        elif player in [player_list[1], player_list[3]]: # atacants
            # Triem les estadístiques més recents dels jugadors
            player_stats = stats_xr.isel(matchday=-1).sel(player=player)[considered_stats_attack].to_array().values
        stats_match = stats_match + list(player_stats) # adjuntem les estadístiques del jugador a les dades d'aquest partit

# Tornem a estandaritzar els valors d'acord a com hem fet amb els valors d'entrenament
stats_match = np.array([stats_match]) # scaler espera una matriu 2D
stats_match_stand = scaler.transform(stats_match.astype(float))

# Fem la predicció
score_prediction = model.predict(stats_match_stand)
score_prediction_output = score_prediction.argmax(axis = 1)

score_prediction_output_label = encoder_scores.inverse_transform(score_prediction_output)

print('Predicted result: ', score_prediction_output_label)

C:\users\admin\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
Predicted result:  ['2-3']
